In [1]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

$$
f(x) = \frac{1}{2}x^T Q x + q^T x
$$
其中：
- \( Q \) 是对称正定矩阵，表示二次项系数矩阵。
- \( q \) 是线性项向量。

约束条件
该问题包含线性不等式约束：
$$
Gx \geq h
$$
其中：
- \( G \) 是约束矩阵。
- \( h \) 是约束的右侧向量。

### 一个示例

- 二次项矩阵：
  $$
  Q = \begin{bmatrix}
  2 & 0 \\
  0 & 2
  \end{bmatrix}
  $$
- 线性项向量：
  $$
  q = \begin{bmatrix}
  -2 \\
  -5
  \end{bmatrix}
  $$
- 不等式约束矩阵：
  $$
  G = \begin{bmatrix}
  1 & -2 \\
  -1 & -2 \\
  -1 & 2 \\
  1 & 0 \\
  0 & 1
  \end{bmatrix}
  $$
- 不等式约束右侧向量：
  $$
  h = \begin{bmatrix}
  -2 \\
  -6 \\
  -2 \\
  0 \\
  0
  \end{bmatrix}
  $$

#### barrier 及 penalty

目标函数：
$$
f(x) = \frac{1}{2}x^T Q x + q^T x + \rho \sum_{i=1}^{m} \max(0, h_i - (Gx)_i)^2
$$
其中：
- $x$ 是优化变量。
- $Q$ 是二次项矩阵。
- $q$ 是线性项向量。
- $Gx \geq h$ 表示不等式约束。
- $\rho$ 是惩罚系数。
- $\max(0, h_i - (Gx)_i)$ 计算约束违反值（仅对违反的约束施加惩罚）。

罚函数的目标函数对违反约束的部分进行平方和惩罚。


目标函数：
$$
f(x) = \frac{1}{2}x^T Q x + q^T x - \mu \sum_{i=1}^{m} \log((Gx)_i - h_i)
$$
其中：
- $ x $ 是优化变量。
- $ Q $ 是二次项矩阵。
- $ q $ 是线性项向量。
- $ Gx \geq h $ 表示不等式约束。
- $ \mu $ 是屏障参数（控制屏障项的权重）。
- $\log((Gx)_i - h_i)$ 表示屏障函数（约束越接近违反，其值越大）。

屏障法的目标函数通过对接近约束边界的变量施加负无穷大代价，防止违反约束。


### scipy

In [4]:
# 定义二次规划参数
Q = np.array([[2, 0], [0, 2]])  # 二次项矩阵
q = np.array([-2, -5])          # 线性项向量
G = np.array([
    [1, -2],
    [-1, -2],
    [-1, 2],
    [1, 0],
    [0, 1]
])  
# 不等式约束矩阵 Gx >= h
h = np.array([-2, -6, -2, 0, 0])  # 不等式约束右侧向量

In [5]:
# 定义二次目标函数
def objective(x, Q, q):
    return 0.5 * np.dot(x, Q @ x) + np.dot(q, x)

# 屏障法
def barrier_method(Q, q, G, h, mu, x0, tol=1e-6, max_iter=50):
    def barrier_objective(x):
        barrier = -mu * np.sum(np.log(G @ x - h))
        return objective(x, Q, q) + barrier

    x = x0.copy()
    for i in range(max_iter):
        result = minimize(barrier_objective, x, method='SLSQP', tol=tol)
        if not result.success:
            print(f"屏障法在第 {i+1} 次迭代中失败: {result.message}")
            break
        x_new = result.x
        # 检查收敛性
        if np.linalg.norm(x_new - x) < tol:
            x = x_new
            print(f"屏障法在第 {i+1} 次迭代中收敛.")
            break
        x = x_new
        mu /= 2  # 减小mu
    return x, result.fun

# 惩罚法
def penalty_method(Q, q, G, h, rho, x0, tol=1e-6, max_iter=50):
    def penalty_objective(x):
        penalties = np.sum(np.maximum(0, h - G @ x) ** 2)
        return objective(x, Q, q) + rho * penalties

    x = x0.copy()
    for i in range(max_iter):
        result = minimize(penalty_objective, x, method='SLSQP', tol=tol)
        if not result.success:
            print(f"惩罚法在第 {i+1} 次迭代中失败: {result.message}")
            break
        x_new = result.x
        # 检查收敛性
        if np.linalg.norm(x_new - x) < tol:
            x = x_new
            print(f"惩罚法在第 {i+1} 次迭代中收敛.")
            break
        x = x_new
        rho *= 2  # 增大rho
    return x, result.fun

In [6]:
# 初始猜测
x0 = np.array([0.5, 0.5])

# 应用屏障法
mu_init = 10.0
x_barrier, f_barrier = barrier_method(Q, q, G, h, mu_init, x0)

# 应用惩罚法
rho_init = 1.0
x_penalty, f_penalty = penalty_method(Q, q, G, h, rho_init, x0)

# 比较结果
results = {
    "屏障法": {
        "最优解 x": x_barrier,
        "目标函数值": f_barrier
    },
    "惩罚法": {
        "最优解 x": x_penalty,
        "目标函数值": f_penalty
    },
}

# 创建DataFrame并显示结果
results_df = pd.DataFrame(results).T
print("屏障法与惩罚法的比较结果：")
print(results_df)


屏障法在第 22 次迭代中收敛.
惩罚法在第 18 次迭代中收敛.
屏障法与惩罚法的比较结果：
                                       最优解 x     目标函数值
屏障法  [1.3996591205227056, 1.699824056646146] -6.449948
惩罚法   [1.3997657479185002, 1.69988438811393] -6.450001


/tmp/ipykernel_9910/244663549.py:8: RuntimeWarning: invalid value encountered in log
  barrier = -mu * np.sum(np.log(G @ x - h))


### cvxpy

In [10]:
import cvxpy as cp

# 定义变量
x = cp.Variable(2)
# 定义目标函数
objective = cp.Minimize(0.5 * cp.quad_form(x, Q) + q @ x)
# 定义约束条件
constraints = [G @ x >= h]
# 求解问题
prob = cp.Problem(objective, constraints)
prob.solve()
print("最优解:", x.value)
print("目标函数值:", prob.value)

最优解: [1.4 1.7]
目标函数值: -6.449999999999999
